## Plots cell2location deconvolution across cma (late vs early paed)

In [1]:
import sys 
import os
from datetime import datetime
date = datetime.now().strftime("%Y-%m-%d")
import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
import hdf5plugin

# Import key analysis packages
import scvi


# Define repo path and add it to sys path (allows to access scripts and metadata from repo)
repo_path = '/nfs/team205/vk8/projects/thymus_ageing_atlas/Spatial_analyses'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/scripts')


In [2]:
# Define paths for plot and path dirs
plot_path = os.path.join(repo_path, 'plots')
data_path = os.path.join(repo_path, 'data')
results_path = os.path.join(repo_path, 'results')
model_path = os.path.join(repo_path, 'models')
general_data_path = '/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/data'

In [3]:
# Set pandas display options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to show output from all the lines in a cells
pd.set_option('display.max_column',None) # display all the columns in pandas
pd.options.display.max_rows = 100

In [4]:
%load_ext autoreload
%autoreload 2
# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
#from plotting.utils import plot_grouped_boxplot, calc_figsize

In [5]:
# Set plot formatting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from matplotlib import rcParams
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text

In [6]:
sc.settings.set_figure_params(dpi = 80, color_map = 'viridis')
#plt.style.use('/nfs/team205/vk8/projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

In [7]:
results_folder = f'{model_path}/ThyAgeing_Visium_c2l_deconvolutions_2025-09-01'
#run_name = f'{results_folder}/{age_group_name}-{suffix}'

## Load reference object

In [8]:
vis_epaed_ffpe = sc.read_h5ad(f'{results_folder}/paed_early-v2-FFPE/adata_vis.zarr')
vis_epaed_oct = sc.read_h5ad(f'{results_folder}/paed_early-v2-OCT/adata_vis.zarr')

vis_lpaed_ffpe = sc.read_h5ad(f'{results_folder}/paed_late-FFPE/adata_vis.zarr')
vis_lpaed_oct = sc.read_h5ad(f'{results_folder}/paed_late-OCT/adata_vis.zarr')

In [9]:
def visium_add_metadata (adata_input, add_c2l, add_histo_anno = {'path':None, 'anno_name': None, 'cma_name': 'manual_bin_cma_3p', 'format_cma': True, 'format_anno': None}):
    if add_c2l:
        adata_input.obs[adata_input.uns['mod']['factor_names']] = adata_input.obsm['q05_cell_abundance_w_sf']
        adata_input.obs['tot_cell_abundance'] = adata_input.uns["mod"]["post_sample_means"]["w_sf"].sum(1).flatten()
        adata_input.obs['detection_sensit']  = adata_input.uns["mod"]["post_sample_q05"]["detection_y_s"]
    if add_histo_anno['path']!= None:
        annot = pd.read_csv(add_histo_anno['path'], index_col=0)
        adata_input.obs = adata_input.obs.merge(annot, left_index=True, right_index=True, how='left').copy()
        if add_histo_anno['format_cma']:
            ct_order = ['Capsular','Sub-Capsular','Cortical level 1','Cortical level 2','Cortical level 3','Cortical CMJ','Medullar CMJ',
            'Medullar level 1','Medullar level 2','Medullar level 3']
            adata_input.obs.loc[adata_input.obs[add_histo_anno['anno_name']]=='Edge', add_histo_anno['cma_name']] = 'Capsular'
            adata_input.obs[add_histo_anno['cma_name']] = adata_input.obs[add_histo_anno['cma_name']].astype('category')
            adata_input.obs[add_histo_anno['cma_name']] = adata_input.obs[add_histo_anno['cma_name']].cat.reorder_categories(ct_order)

        if add_histo_anno['format_anno'] != None:
            adata_input.obs[add_histo_anno['anno_name']] = adata_input.obs[add_histo_anno['anno_name']].astype('category')
            adata_input.obs[add_histo_anno['anno_name']] = adata_input.obs[add_histo_anno['anno_name']].cat.reorder_categories(add_histo_anno['format_anno'])
        

In [10]:
visium_add_metadata (adata_input = vis_epaed_ffpe, add_c2l = True, 
                     add_histo_anno = {'path':f'{results_path}/ThyAge_Visium_epaed_FFPE_annos_merged_2025-09-08.csv', 'anno_name': 'annotations_v2', 'cma_name': 'manual_bin_cma_3p', 'format_cma': True, 'format_anno': None})

In [11]:
np.mean(vis_epaed_ffpe.obs['manual_bin_cma_3p'].isnull())

0.231274246493584

In [12]:
visium_add_metadata (adata_input = vis_epaed_oct, add_c2l = True, 
                     add_histo_anno = {'path':f'{results_path}/ThyAge_Visium_epaed_OCT_annos_merged_2025-09-14.csv', 'anno_name': 'annotations_level_0', 'cma_name': 'manual_bin_cma_3p', 'format_cma': True, 'format_anno': None})

In [13]:
np.mean(vis_epaed_oct.obs['manual_bin_cma_3p'].isnull())

0.16186899196301938

In [14]:
visium_add_metadata (adata_input = vis_lpaed_oct, add_c2l = True, 
                     add_histo_anno = {'path':f'{results_path}/ThyAge_Visium_OCT_FFPE_annos_merged_2025-09-05.csv', 'anno_name': 'annotations_v1', 'cma_name': 'manual_bin_cma_3p', 'format_cma': True, 'format_anno': None})

/tmp/ipykernel_1370843/90477010.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  annot = pd.read_csv(add_histo_anno['path'], index_col=0)


In [15]:
visium_add_metadata (adata_input = vis_lpaed_ffpe, add_c2l = True, 
                     add_histo_anno = {'path':f'{results_path}/ThyAge_Visium_OCT_FFPE_annos_merged_2025-09-05.csv', 'anno_name': 'annotations_v2', 'cma_name': 'manual_bin_cma_3p', 'format_cma': True, 'format_anno': None})

/tmp/ipykernel_1370843/90477010.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  annot = pd.read_csv(add_histo_anno['path'], index_col=0)


In [16]:
cma = ['manual_bin_cma_3p']

In [17]:
np.mean(vis_lpaed_oct.obs[cma].isnull())
np.mean(vis_lpaed_ffpe.obs[cma].isnull())

0.25761457443780245

0.1513030528667163

cell_groups = {'T_NK': lymphoid,
               'TEC': tecs,
               'Stroma': stroma,
               'B_Myeloid': immune}

## Plot cell type abundance in CMA bins

In [21]:
vis_all = ad.concat([vis_epaed_oct, vis_epaed_ffpe, vis_lpaed_oct, vis_lpaed_ffpe], merge = "unique", uns_merge = "unique")

In [23]:
factor_ctypes = list(np.intersect1d(vis_epaed_oct.uns['mod']['factor_names'].tolist(), vis_lpaed_oct.uns['mod']['factor_names'].tolist()))

In [97]:
vis_all_filt = vis_all[vis_all.obs['total_counts']>1000,].copy()

In [24]:
cellab_vis_all = sc.AnnData(vis_all_filt.obs[factor_ctypes], 
                         obs = vis_all_filt.obs.drop(factor_ctypes, axis = 1),obsm = vis_all_filt.obsm)

sc.pp.normalize_total(cellab_vis_all,target_sum=100)


In [25]:
cell_groups = {'T_early': [['T_DN(early)', 'T_DN(P)', 'T_DN(Q)'], [0.25]],
               'T_traj':[['T_DN(P)','T_DN(Q)', 'T_DN(late)','T_DP(P)','T_DP(Q)', 'T_αβT(entry)', 'T_CD8_naive', 'T_CD4_naive'],[2.0]],
               'Treg' :[['T_Treg(agonist)','T_Treg','T_Treg_tr', 'T_Treg_recirc'], [1.0]], 
               'Trecirc': [['T_CD8_naive_recirc', 'T_CD8_em', 'T_CD8_rm', 'T_CD8_age-assoc','T_CD4_act', 'T_CD4_naive_recirc', 'T_CD4_h'], [1.0]],
               'CD8AA': [['T_CD8αα(entry)', 'T_CD8αα(I)','T_CD8αα(II)'],[0.5]], 
               'innate': [['T_MAIT', 'ILC', 'T_γδT', 'NK_tr'],[0.5]], 
               'TEC-major': [['cTECIII','cTECII', 'cTECI','mcTEC', 'mTECI', 'mTECII','mTECIII'],[0.5]], 
               'TEC-mimetic': [['mTECI-trans', 'TEC-myo', 'TEC-neuro', 'TEC-cilliated', 'TEC-iono', 'TEC-tuft', 'TEC-EMT'],[0.25]], 
                'B cells': [['B_dev_thy','B_naive', 'B_mem','B_age-associated', 'B_GC-like','B_plasma', 'B_plasmablast'], [0.5]], 
               'Stroma': [['Fb-interlo_PI16', 'Fb-interlo','Fb-perilo','Fb-perilo_COLEC11','Fb-med','Fb-interm','Fb-adipo','Adipocyte','EC-art',
 'EC-cap','EC-cap_lipid','EC-ven','EC-peri','EC-lymphatic','SMC-art','Peri-cap','Peri-smc-interm','SMC-ven'],[0.5]], 
               'Myeloid': [['CMP', 'Promonocyte','Mono','Mac', 'Mac_LYVE1', 'Mac_SPIC', 'Mast'],[0.5]], 
               'DCs': [['DC1', 'DC2', 'DC2_SFTPD', 'aDC2','aDC3','pDC'], [0.5]]}

In [26]:
for ctype_group, ctypes in cell_groups.items():
    var_names = [i for i in ctypes[0] if i in factor_ctypes]
    ## Prepare epaed age group
    ad_early =  cellab_vis_all[((cellab_vis_all.obs['age_group'] == 'paed_early') & (cellab_vis_all.obs['chemistry_simple'] == 'Visium_OCT')), var_names]
    epaed_df_full = pd.DataFrame(ad_early.X, 
                                 index = ad_early.obs['manual_bin_cma_3p'], columns = var_names)
    epaed_df_full.values[epaed_df_full.values<ctypes[1][0]] = 0
    epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
    ## Prepare lpaed age group
    ad_late =  cellab_vis_all[((cellab_vis_all.obs['age_group'] == 'paed_late') & (cellab_vis_all.obs['chemistry_simple'] == 'Visium_OCT')), var_names] 
    lpaed_df_full = pd.DataFrame(ad_late.X, 
                                    index = ad_late.obs['manual_bin_cma_3p'], columns = var_names)
    lpaed_df_full.values[lpaed_df_full.values<ctypes[1][0]] = 0
    lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
    norm_values = np.maximum(epaed_df_summary.max().values, lpaed_df_summary.max().values)
    epaed_df_color = epaed_df_summary.div(norm_values, axis = 1)
    lpaed_df_color = lpaed_df_summary.div(norm_values, axis = 1)
    # Calculate dot size
    max_dot_epaed = np.max(epaed_df_full.groupby('manual_bin_cma_3p').agg(lambda x: np.mean(x>0)))
    max_dot_lpaed = np.max(lpaed_df_full.groupby('manual_bin_cma_3p').agg(lambda x: np.mean(x>0)))
    max_dot = round(max(max_dot_epaed, max_dot_lpaed), 1)

    # --- 3) Plot both panels from the normalized layer on [0,1] ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    sc.pl.dotplot(adata = ad_early,dot_color_df = epaed_df_color, 
        var_names=var_names, groupby='manual_bin_cma_3p', cmap='viridis',
        title=f'Epaed OCT:{ctype_group}, cut-off {ctypes[1][0]}',
        expression_cutoff=ctypes[1][0],
        vmin=0.0, vmax=1.0, dot_max= max_dot,
        ax=axes[0], show=False)

    sc.pl.dotplot(adata = ad_late,dot_color_df = lpaed_df_color, 
        var_names=var_names, groupby='manual_bin_cma_3p', cmap='viridis',
        title=f'Lpaed OCT:{ctype_group}, cut-off {ctypes[1][0]}',
         expression_cutoff=ctypes[1][0],
        vmin=0.0, vmax=1.0, dot_max=max_dot,
        ax=axes[1], show=False
    )

    plt.tight_layout()
    plt.savefig(
        f'{results_folder}/plots/{ctype_group}_epaed_vs_lpaed_OCT_cutoff{ctypes[1][0]}_pervar-norm.pdf',
        bbox_inches='tight'
    )
    plt.close(fig)

/tmp/ipykernel_1339598/4095104990.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/tmp/ipykernel_1339598/4095104990.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

...

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/tmp/ipykernel_1339598/4095104990.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/4095104990.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

In [27]:
for ctype_group, ctypes in cell_groups.items():
    var_names = [i for i in ctypes[0] if i in factor_ctypes]
    ## Prepare epaed age group
    ad_early =  cellab_vis_all[((cellab_vis_all.obs['age_group'] == 'paed_early') & (cellab_vis_all.obs['chemistry_simple'] == 'Visium_FFPE')), var_names]
    epaed_df_full = pd.DataFrame(ad_early.X, 
                                 index = ad_early.obs['manual_bin_cma_3p'], columns = var_names)
    epaed_df_full.values[epaed_df_full.values<ctypes[1][0]] = 0
    epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
    ## Prepare lpaed age group
    ad_late =  cellab_vis_all[((cellab_vis_all.obs['age_group'] == 'paed_late') & (cellab_vis_all.obs['chemistry_simple'] == 'Visium_FFPE')), var_names] 
    lpaed_df_full = pd.DataFrame(ad_late.X, 
                                    index = ad_late.obs['manual_bin_cma_3p'], columns = var_names)
    lpaed_df_full.values[lpaed_df_full.values<ctypes[1][0]] = 0
    lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
    norm_values = np.maximum(epaed_df_summary.max().values, lpaed_df_summary.max().values)
    epaed_df_color = epaed_df_summary.div(norm_values, axis = 1)
    lpaed_df_color = lpaed_df_summary.div(norm_values, axis = 1)
    # Calculate dot size
    max_dot_epaed = np.max(epaed_df_full.groupby('manual_bin_cma_3p').agg(lambda x: np.mean(x>0)))
    max_dot_lpaed = np.max(lpaed_df_full.groupby('manual_bin_cma_3p').agg(lambda x: np.mean(x>0)))
    max_dot = round(max(max_dot_epaed, max_dot_lpaed), 1)

    # --- 3) Plot both panels from the normalized layer on [0,1] ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    sc.pl.dotplot(adata = ad_early,dot_color_df = epaed_df_color, 
        var_names=var_names, groupby='manual_bin_cma_3p', cmap='viridis',
        title=f'Epaed FFPE:{ctype_group}, cut-off {ctypes[1][0]}',
        expression_cutoff=ctypes[1][0],
        vmin=0.0, vmax=1.0, dot_max= max_dot,
        ax=axes[0], show=False)

    sc.pl.dotplot(adata = ad_late,dot_color_df = lpaed_df_color, 
        var_names=var_names, groupby='manual_bin_cma_3p', cmap='viridis',
        title=f'Lpaed FFPE:{ctype_group}, cut-off {ctypes[1][0]}',
         expression_cutoff=ctypes[1][0],
        vmin=0.0, vmax=1.0, dot_max=max_dot,
        ax=axes[1], show=False
    )

    plt.tight_layout()
    plt.savefig(
        f'{results_folder}/plots/{ctype_group}_epaed_vs_lpaed_FFPE_cutoff{ctypes[1][0]}_pervar-norm.pdf',
        bbox_inches='tight'
    )
    plt.close(fig)

/tmp/ipykernel_1339598/1941285413.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/tmp/ipykernel_1339598/1941285413.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

...

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/tmp/ipykernel_1339598/1941285413.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  epaed_df_summary = epaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lpaed_df_summary = lpaed_df_full.groupby('manual_bin_cma_3p').mean()
/tmp/ipykernel_1339598/1941285413.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dot_epaed = np.max(epa

{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

In [ ]:
for ctype_group, ctypes in cell_groups.items():
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    sc.pl.dotplot(vis_all_filt[(vis_all_filt.obs['age_group'] == 'paed_early') & (vis_all_filt.obs['chemistry_simple'] == 'Visium_OCT'),], 
                  var_names=[i for i in ctypes[0] if i in factor_ctypes], groupby='manual_bin_cma_3p', cmap='viridis',
                  title=f'Epaed OCT:{ctype_group}, cut-off {ctypes[1][0]}', expression_cutoff=ctypes[1][0], standard_scale='var', ax=axes[0], show=False)
    sc.pl.dotplot(vis_all_filt[(vis_all_filt.obs['age_group'] == 'paed_late') & (vis_all_filt.obs['chemistry_simple'] == 'Visium_OCT'),], 
                  var_names=[i for i in ctypes[0] if i in factor_ctypes], groupby='manual_bin_cma_3p', cmap='viridis',
                  title=f'Lpaed OCT:{ctype_group}, cut-off {ctypes[1][0]}', expression_cutoff=ctypes[1][0], standard_scale='var', ax=axes[1], show=False)
    plt.tight_layout()
    plt.savefig(f'{results_folder}/plots/{ctype_group}_epaed_vs_lpaed_OCT_cutoff{ctypes[1][0]}.pdf', bbox_inches='tight')
    plt.close(fig)

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


...

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


{'mainplot_ax': <Axes: >,
 'size_legend_ax': <Axes: title={'center': 'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <Axes: title={'center': 'Mean expression\nin group'}>}

/nfs/team205/vk8/mambaforge/envs/c2l_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
